# Preparation

### Import

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D
from tensorflow.keras import Model

### Load DATA

In [ ]:
def three_images_per_second(video_file, output_dir, fps):
    cap = cv2.VideoCapture(video_file)
    frame_num = 0
    image_num = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            if frame_num % (fps // 3) == 0:
                cv2.imwrite(output_dir+"/image{}.jpg".format(image_num),frame)
                image_num += 1
        else:
            break
        frame_num += 1

    cap.release()
    cv2.destroyAllWindows()
    return

In [ ]:
three_images_per_second('StreetCamera.mp4', 'StreetCamera', 100)